<a href="https://colab.research.google.com/github/CamilaCSilva/C318/blob/main/C318.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


IMPORTS


In [ ]:
pip install seaborn

In [2]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv

In [3]:
URL_PS_VENDAS_2019 = 'bancodados/201912.csv'
URL_ARQUIVO_GEOGRAFICO = "bancodados/geografia.csv"
URL_ARQUIVO_CARACTERISTICAS_PRODUTOS = "bancodados/caracteristicas_produtos_saude_suplementar.csv"
URL_ARQUIVO_LAT_LONG = "bancodados/latitude-longitude-cidades.csv"
URL_ARQUIVO_HISTORICO = "bancodados/HISTORICO_PLANOS.csv"

LEITURA DOS DATASETS

In [ ]:
tabela_ps_vendas_2019 = pd.read_csv(URL_PS_VENDAS_2019, delimiter=";", low_memory=False, skiprows=lambda x: x > 0 and random.random() >= 0.1, quoting=csv.QUOTE_NONE, encoding='utf-8')
#, skiprows=lambda x: x > 0 and random.random() >= 0.001

#preenche campos com valor NaN com 0
tabela_ps_vendas_2019.fillna(0, inplace=True)

#tira as aspas das colunas e registros e transforma em int e float
tabela_ps_vendas_2019.columns = tabela_ps_vendas_2019.columns.str.replace('"', '')
tabela_ps_vendas_2019['CD_MUNICIPIO'] =  tabela_ps_vendas_2019['CD_MUNICIPIO'].str.replace('"', '').astype(int)
tabela_ps_vendas_2019['ID_PLANO'] =  tabela_ps_vendas_2019['ID_PLANO'].str.replace('"', '').astype(int)
tabela_ps_vendas_2019['CD_FAIXA_ETARIA'] =  tabela_ps_vendas_2019['CD_FAIXA_ETARIA'].str.replace('"', '').astype(int)
tabela_ps_vendas_2019['VCM'] =  tabela_ps_vendas_2019['VCM'].str.replace('"', '').str.replace(',', '.').astype(float)

tabela_ps_vendas_2019

In [ ]:
tabela_geografica = pd.read_csv(URL_ARQUIVO_GEOGRAFICO, sep=";")
tabela_geografica_simplificada = tabela_geografica[['CD_MUNICIPIO', 'NM_MUNICIPIO', 'SG_UF', 'NM_REGIAO']]
tabela_geografica_simplificada

In [ ]:
tabela_caracteristicas = pd.read_csv(URL_ARQUIVO_CARACTERISTICAS_PRODUTOS, delimiter=";", quoting=csv.QUOTE_NONE, encoding='utf-8')
tabela_caracteristicas_simplificada = tabela_caracteristicas[['ID_PLANO', 'CD_PLANO', 'NM_PLANO', 'CD_OPERADORA', 'GR_MODALIDADE',
                                                              'VIGENCIA_PLANO', 'GR_CONTRATACAO', 'SGMT_ASSISTENCIAL', 'COBERTURA',
                                                              'ABRANGENCIA_COBERTURA', 'ACOMODACAO_HOSPITALAR', 'SITUACAO_PLANO',
                                                              'DT_REGISTRO_PLANO']]

tabela_caracteristicas_simplificada

MERGE DOS DATASETS

In [ ]:
tabela_lat_long = pd.read_csv(URL_ARQUIVO_LAT_LONG, delimiter=',')
tabela_lat_long_simplificada = tabela_lat_long[['NM_MUNICIPIO', 'latitude', 'longitude']]
tabela_lat_long_simplificada

In [ ]:
tabela_merge_vendas_geografia = pd.merge(tabela_ps_vendas_2019, tabela_geografica_simplificada, how='outer', on='CD_MUNICIPIO')
tabela_merge_vendas_geografia

In [ ]:
tabela_merge_vendas_geografia_caracteristicas = pd.merge(tabela_merge_vendas_geografia, tabela_caracteristicas, how='outer', on='ID_PLANO')
tabela_merge_vendas_geografia_caracteristicas

In [ ]:
tabela_merge_completa = pd.merge(tabela_merge_vendas_geografia_caracteristicas, tabela_lat_long_simplificada, how='outer', on='NM_MUNICIPIO')
tabela_merge_completa

FILTRAGEM

In [47]:
tabela_completa_filtrada = tabela_merge_completa[tabela_merge_completa['SITUACAO_PLANO'] == 'Ativo'] #108769
tabela_completa_filtrada.fillna(0, inplace=True)
tabela_completa_filtrada = tabela_completa_filtrada[tabela_completa_filtrada['VCM'] != 0]
tabela_completa_filtrada

<ipython-input-47-3c4e39dc0573>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabela_completa_filtrada.fillna(0, inplace=True)


,CD_MUNICIPIO,ID_PLANO,CD_FAIXA_ETARIA,VCM,NM_MUNICIPIO,SG_UF,NM_REGIAO,CD_PLANO,NM_PLANO,CD_OPERADORA,...,ABRANGENCIA_COBERTURA,FATOR_MODERADOR,ACOMODACAO_HOSPITALAR,LIVRE_ESCOLHA,SITUACAO_PLANO,DT_SITUACAO,DT_REGISTRO_PLANO,DT_ATUALIZACAO,latitude,longitude
1,110001.0,1329193.0,8.0,2187.78,Alta Floresta D'Oeste,RO,Norte,441999035,"""Bradesco Saúde Top NPlus Q CE A""",5711.0,...,Nacional,Ausente,Individual,Parcial com internação,Ativo,2003-03-27,2003-03-27,2023-12-06,-11.9283,-61.9953
2,110001.0,1329198.0,3.0,971.37,Alta Floresta D'Oeste,RO,Norte,442000034,"""Bradesco Saúde Top NPlus Q CE copart A I""",5711.0,...,Nacional,Coparticipacão,Individual,Parcial com internação,Ativo,2003-03-27,2003-03-27,2023-12-06,-11.9283,-61.9953
3,110001.0,1329198.0,6.0,1368.72,Alta Floresta D'Oeste,RO,Norte,442000034,"""Bradesco Saúde Top NPlus Q CE copart A I""",5711.0,...,Nacional,Coparticipacão,Individual,Parcial com internação,Ativo,2003-03-27,2003-03-27,2023-12-06,-11.9283,-61.9953
4,110001.0,1329696.0,5.0,1483.20,Alta Floresta D'Oeste,RO,Norte,445620033,"""Bradesco Saúde Top NPlus Q CE B I""",5711.0,...,Nacional,Ausente,Individual,Parcial com internação,Ativo,2003-09-05,2003-09-05,2023-12-06,-11.9283,-61.9953
5,110001.0,1329696.0,6.0,1527.69,Alta Floresta D'Oeste,RO,Norte,445620033,"""Bradesco Saúde Top NPlus Q CE B I""",5711.0,...,Nacional,Ausente,Individual,Parcial com internação,Ativo,2003-09-05,2003-09-05,2023-12-06,-11.9283,-61.9953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128321,110145.0,16676441.0,9.0,923.23,Parecis,RO,Norte,476396163,"""Exato Empresarial/PME Trad.15 AH QP COP""",6246.0,...,Nacional,Coparticipacão,Individual,Parcial com internação,Ativo,2019-11-22,2016-07-25,2023-12-06,-12.1754,-61.6032
128325,110145.0,1403986.0,4.0,498.07,Parecis,RO,Norte,408997999,"""UNIMED Total Nacional Coletivo Empresarial""",347507.0,...,Nacional,Ausente,Coletiva,Ausente,Ativo,2023-01-30,1999-02-25,2023-12-06,-12.1754,-61.6032
128326,110145.0,1403986.0,7.0,745.83,Parecis,RO,Norte,408997999,"""UNIMED Total Nacional Coletivo Empresarial""",347507.0,...,Nacional,Ausente,Coletiva,Ausente,Ativo,2023-01-30,1999-02-25,2023-12-06,-12.1754,-61.6032
128329,110145.0,16132936.0,5.0,423.71,Parecis,RO,Norte,466997125,"""UNIMED IDEAL ESPECIAL ESTADUAL COLETIVO EMPRE...",347507.0,...,Estadual,Coparticipacão,Individual,Ausente,Ativo,2012-06-04,2012-06-04,2023-12-06,-12.1754,-61.6032


In [ ]:
#tabela_merge_completa[tabela_merge_completa['SITUACAO_PLANO'] == 'Suspenso'] #73285
#tabela_merge_completa[tabela_merge_completa['SITUACAO_PLANO'] == 'Cancelado'] #70002
#tabela_merge_completa[tabela_merge_completa['SITUACAO_PLANO'] == 'Transferido'] #25541
#tabela_merge_completa[tabela_merge_completa['SITUACAO_PLANO'] != 'Ativo'] #174932 planos não ativos
#tabela_teste = tabela_merge_completa[tabela_merge_completa['SITUACAO_PLANO'] == 'Ativo']
#tabela_teste.fillna(0, inplace=True)
#tabela_teste[tabela_teste['VCM'] == 0] #24050 registros sem o preco do plano

ANALISES

In [50]:
print('Número de tipos de planos:')
tabela_completa_filtrada['ID_PLANO'].count()

Número de tipos de planos:


84719